In [1]:
import numpy as np
import onnx
import os
import glob
from onnx import numpy_helper
import tvm
from tvm import te
import tvm.relay as relay

In [2]:
# module path
mobilenet_path = "../mobilenet/mobilenet_v2_1.0_224.tflite"
tflite_module_buf = open(mobilenet_path, "rb").read()

In [3]:
# load module param and mod
try:
    import tflite
    
    mobilenet_model = tflite.Model.GetRootAsModel(tflite_module_buf, 0)
except AttributeError:
    import tflite.Model
    
    mobilenet_model = tflite.Model.Model.GetRootAsModel(tflite_module_buf, 0)
# mobilenet_model = onnx.load(mobilenet_path)
input_name = "input"
input_shape = (1, 224, 224, 3)
input_type = "float32"

shape_dict = {input_name: input_shape}
print(shape_dict)
dtype_dict = {input_name: input_type}
print(dtype_dict)

{'input': (1, 224, 224, 3)}
{'input': 'float32'}


In [4]:
# mod, params = relay.frontend.from_onnx(mobilenet_model, shape=shape_dict, dtype=dtype_dict)
mod, params = relay.frontend.from_tflite(mobilenet_model, shape_dict=shape_dict, dtype_dict=dtype_dict)

In [5]:
# target = tvm.target.arm_cpu("rasp3b")
# target = tvm.target.Target('llvm')
target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
# target = tvm.target.intel_graphics()
# target = tvm.target.Target("opencl -device=intel_graphics") -device=mali
# target = tvm.target.Target("opencl")
print(target)

llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -mtriple=aarch64-linux-gnu


In [6]:
target_host = tvm.target.Target('llvm -device=arm_cpu -mtriple=aarch64-linux-gnu')
# target_host = tvm.target.Target('llvm')
print(target_host)

llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mtriple=aarch64-linux-gnu


In [7]:
with tvm.transform.PassContext(opt_level=3):   
    module_lib = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mobilenet')
print(module_lib)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [8]:
# output lib path
lib_path = "../module/armv8/mobilenet.so"

In [9]:
# export params path
param_path = "../module/armv8/mobilenet.params"

In [10]:
module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler("aarch64-linux-gnu-g++"))
# cc="/usr/bin/aarch64-linux-gnu-g++"
# cc="/usr/bin/arm-linux-gnueabihf-g++
# tvm.contrib.cc.cross_compiler("aarch64-linux-gnu-g++")

In [11]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))